##**BIO/QTM 385: In class exercise for Wednesday, October 27th** 


(answers will be the part of Assignment #4, Due 11/3)

As always, all questions to be answered will be in <font color="blue"> blue</font> and places to write your answers will be in <font color="green"> green</font>.

In [ ]:
#import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
import numpy.linalg as linalg
import scipy.optimize as optimize
from scipy.optimize import least_squares

In [ ]:
#@title Figure Settings
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/NMA2020/nma.mplstyle")

##The Fisher Information Matrix and Sloppy Models

As we've seen several times by now, when we're trying to infer the parameters ($\theta\in \mathcal{R}^p$) of a model, $\mathcal{M}$, based on data, $X$, we can calculate the *posterior distribution* of our parameter values given the the data via:
\begin{equation}
p(\theta \vert X) \propto p(X \vert \theta) p(\theta),
\end{equation}
where $p(X \vert \theta)$ is the *likelihood* of observing the data given a particular set of parameter values and $p(\theta)$ is our *prior* distribution over parameter space (we'll ignore the normalization from $p(X)$ for now).

As we saw in class, though, often the likelihood function often is not able to precisely specify one or many of the model's parameters.  We can quantify this notion using the Fisher Information Matrix (FIM) - which we will refer to as $g$.  In this matrix, each entry is the average curvature of the log-likelihood function in their respective parameter directions.  If we have a data set with $N$ data points, and $\theta_i$ and $\theta_j$ are two of the model parameters, then the FIM is given by:
\begin{equation}
g_{i,j} = -\sum_{k=1}^N p(x_k\vert\theta) \frac{\partial^2}{\partial\theta_i\partial\theta_j}\log p(x_k\vert\theta).
\end{equation}

Note how this matrix is symmetric ($\frac{\partial^2 f}{\partial x \partial y}=\frac{\partial^2 f}{\partial y \partial x}$) and real-valued, so it's a positive-definite matrix with (all eigenvales are greater than zero).  Thus, we can look at the eigenvalue structure.  

Here, large eigenvalues correspond to eigenvectors in parameter space that have a large curvature in the log-likelihood function - implying that we can perform inference well.  Small eigenvalues correspond to eigenvectors in parameter space that have small curvatures - implying that it will be very difficult to infer parameters along those directions.

In the case of least-squares fitting to a model function $f(t;\theta)$, we can define the FIM using:
\begin{equation}
g_{i,j} = \sum_{k=1}^N \frac{\partial f(t_k;\theta)}{\partial \theta_i}\frac{\partial f(t_k;\theta)}{\partial \theta_j} = (J^T(\theta) J(\theta))_{i,j},
\end{equation}
where $J(\theta)$ is the Jacobian matrix ($J_{k,i} = \frac{\partial f(t_k;\theta)}{\partial \theta_i}$).

##An Example: Mixtures of Exponentials

For this part of the exercise, you will try to infer the parameters from a model of the form:
\begin{equation}
y(t) = \sum_{i=1}^d A_i e^{-t/\tau_i}.
\end{equation}
The code below defines a function, ```multiExponential(t,As,taus)```, that generates outputs, ```y``` for input array ```t``` (and parameter arrays ```As``` and ```taus```), and ```createRandomExponentialModel(d)``` that will create a random model of order ```d``` (as defined in the equation above).

In [ ]:
def multiExponential(t,As,taus):
    y = np.zeros(len(t))
    L = len(As)
    for i in range(L):
        y += As[i]*np.exp(-t/taus[i])
    return y

def createRandomExponentialModel(d,Amax=10,tauMax = 10):
  As = random.uniform(0,Amax,d)
  taus = random.uniform(0,tauMax,d)
  return As,taus

<font color=blue>Question #1: Create a random exponential model with $d=7$ and plot its evaluation ($y$ vs. $t$) over ```np.arange(0,21,1)```.  Looking at this curve, would you expect to be able to infer all 14 parameters?  Why or why not?</font>

In [ ]:
#Type code for question #1 here.  

<font color="green"> Looking at this curve, would you expect to be able to infer all 14 parameters?  Why or why not? </font>

The code below introduces a function, ```returnExponentialCurveFit(t,y,d)```, that fits a given data set (```t``` and ```y```) to the multi-exponential curve of order ```d``` above by minimizing the least squares function.  The output, ```yFit``` is the best-fit curve resulting from the fit.

In [1]:
def leastSquaresFunction_Exponentials(theta,t,y):
  d = int(np.shape(theta)[0]/2)
  As = np.exp(theta[:d])
  taus = np.exp(theta[d:])
  return multiExponential(t,As,taus) - y

def returnExponentialCurveFit(t,y,d):

  A0 = np.log(random.dirichlet(np.ones(d))*y[0])
  tau0 = np.log(random.uniform(0,np.max(t),size=d))
  theta0 = np.zeros(2*d)
  theta0[:d] = A0
  theta0[d:] = tau0

  lsq = least_squares(leastSquaresFunction_Exponentials, theta0, args=(t, y))

  Afit = np.exp(lsq.x[:d])
  tauFit = np.exp(lsq.x[d:])

  yFit = multiExponential(t,Afit,tauFit)

  return yFit

<font color=blue> Question #2: Run ```returnExponentialCurveFit()``` for $d=1, 3, 5,$ and $7$ using your data from Question #1.  Plot the resulting best fit curves (comparing to the original data) for each fit.  Visually (using ```plt.semilogy()```), which value of $d$ would you choose to model the data if you didn't know that it was actually generated from the $d=7$ equation?</font>

In [ ]:
#Type your code for Question #2 here

<font color=green> Visually, which value of $d$ would you choose to model the data if you didn't know that it was actually generated from the $d=7$ equation? </font>

As shown in class, for a least-squares fitting problem with uniform measurement error, $\sigma$, we can write the log-likelihood function via: 
\begin{equation}
\mathcal{L}(x\vert \theta) = -\frac{1}{2}\log\left(2\pi \sigma^2\right) - \frac{1}{2\sigma^2}\chi^2(\theta,y),
\end{equation}
where
\begin{equation}
\chi^2(\theta,y) = \sum_{k=1}^N \left[y_i - \hat{y}(x_i,\theta) \right]^2
\end{equation}
and $\hat{y}(x_i,\theta)$ is the best-fit curve.

<font color=blue> Question #3: Add gaussian noise with standard deviation $\sigma = .1$ to your previous value of ```y``` to make a new variable, ```z```.  Rembering that the BIC = $-2\mathcal{L}(x\vert \theta,\mathcal{M}) + p\log N$, use Bayesian model selection to select the most likely value for $d$ given the data.  Explain your reasoning. Do you get $d=7$?  If not, why might this be? (You can assume that $\sigma$ in the log-likelihood equation above is equal to 0.1)</font>

In [ ]:
#Type code for Question #3 here

<font color = green> Explain your reasoning. Do you get $d=7$?  If not, why might this be?</font>

### The Fisher Information Matrix

For the multi-exponential system, our partial derivatives are given by:
\begin{eqnarray}
\frac{\partial y(t;\theta)}{\partial A_i} &=& e^{-t/\tau_i} \\
\frac{\partial y(t;\theta)}{\partial \tau_i } &=& \frac{A_i t}{\tau_i^2}e^{-t/\tau_i}.
\end{eqnarray}

<font color=blue>Question #4: Write a function ```returnFisherInformationMultiExponentials(t,y,As,taus)``` that returns the Fisher Information Matrix for the multi-exponential model near the "true" value of the model ($d=7$).  Note that the output should be a $2d\times 2d$ matrix.  </font>

In [ ]:
#Type code for Question #4 here
def returnFisherInformationMultiExponentials(t,y,As,taus):

In [ ]:
def makeEigPlot(vals):
    np.sort(vals)[::-1]
    L = np.shape(vals)[0]
    for i in range(L):
        plt.semilogy([0,1],[vals[i],vals[i]],'k-')
    plt.xlim([-.1, 1.1])
    plt.ylabel('Eigenvalues')
    plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,      
    top=False,         
    labelbottom=False) 
    plt.show()

<font color=blue> Question #5: Calculate the eigenvalues and eigenvectors for the FIM using ```t```, ```As``` and ```taus``` from Question #1 and ```z``` from question #2 as your value for $y$.  Use ```makeEigPlot(eigenvalues)``` (in the cell above) to plot the eigenvalue spectrum for your FIM.  Does this model appear to be sloppy?  Is it identifiable?

In [ ]:
#Enter your code for Question #5 here

<font color="green"> Does this model appear to be sloppy?  Is it identifiable? </font>

<font color=blue> Question #6: Visualize the eigenvectors from Question #5 using the method of your choosing.  Would it be easy to eliminate the smallest eigenvectors from your model?  Explain your reasoning.

In [ ]:
#Enter your code for Question #6 here

<font color=green> Would it be easy to eliminate the smallest eigenvectors from your model?  Explain your reasoning. </font>